### 1. Define Structure

In [1]:
import os
import random
import json

import nltk
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class ChatbotModel(nn.Module):

    def __init__(self, input_size, output_size):
        super(ChatbotModel, self).__init__()

        #  fc1: รับข้อมูลจากคลังคำศัพท์ (input_size) แล้วขยาย/แปลงเป็น 128 มิติ
        self.fc1 = nn.Linear(input_size, 128)

        # fc2: บีบข้อมูลจาก 128 เหลือ 64 มิติ เพื่อสรุปใจความสำคัญ
        self.fc2 = nn.Linear(128, 64)

        # fc3: ตัวตัดสินใจสุดท้าย โดยส่งค่าออกไปตามจำนวนหมวดหมู่คำตอบที่มี (output_size)
        self.fc3 = nn.Linear(64, output_size)

        #  relu: ตัวช่วยให้บอทเข้าใจความซับซ้อนของภาษา (Non-linearity)
        self.relu = nn.ReLU()

        # dropout: ตัวช่วยกันบอท "ท่องจำ" โดยสุ่มปิดการทำงานของเซลล์ประสาท 50% ในขณะเทรน PyTorch Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, X):

        # Input -> fc1 -> ReLU: ข้อมูลถูกส่งเข้าเลเยอร์แรกและแปลงค่าด้วย ReLU เพื่อหาความสัมพันธ์ของคำ
        X = self.relu(self.fc1(X))

        # Dropout: สุ่มปิดสัญญาณบางส่วน (เฉพาะตอนเทรน) เพื่อให้โมเดลมีความยืดหยุ่น
        X = self.dropout(X)

        # fc2 -> ReLU: ประมวลผลข้อมูลในระดับที่ลึกขึ้นเพื่อระบุเอกลักษณ์ของประโยค
        X = self.relu(self.fc2(X))

        # Dropout: ทำซ้ำอีกครั้งเพื่อความเสถียร
        X = self.dropout(X)

        # fc3: ส่งผลลัพธ์เป็นคะแนน (Raw scores/Logits) ของแต่ละ Intent ออกไป
        X = self.fc3(X)

        return X


- ข้อมูลจะถูกส่งเข้าเป็นตัวเลข (Bag of Words) -> ถูกบีบอัดและวิเคราะห์ผ่าน 3 เลเยอร์ -> ผลลัพธ์สุดท้ายคือ คะแนนความมั่นใจ ในแต่ละ Tag (เช่น เป็นการทักทาย 90%, เป็นการถามราคา 5%) เพื่อให้ฟังก์ชัน process_message เลือกคำตอบที่แม่นยำที่สุดมาแสดงครับ PyTorch nn.Module Guide

### 2. Define Manager Class

In [ ]:
class ChatbotAssistant:

    def __init__(self, intents_path, function_mappings = None):

        # self.model = None: จองที่ไว้สำหรับเก็บตัวโมเดล Neural Network (PyTorch) 
        # ซึ่งจะถูกสร้างขึ้นภายหลังในขั้นตอนการเทรนหรือการโหลดไฟล์ PyTorch nn.Module Reference
        self.model = None

        # self.intents_path = intents_path: เก็บที่อยู่ (Path) ของไฟล์ JSON 
        # ที่บรรจุโครงสร้างคำถาม-คำตอบ (เช่น intents.json)
        self.intents_path = intents_path

        # self.documents = []: เก็บรายการคู่ของ (รายการคำศัพท์, แท็ก) 
        # เช่น (["hello"], "greeting") เพื่อใช้เป็นฐานข้อมูลในการเทรน
        self.documents = []

        # self.vocabulary = []: เก็บ คำศัพท์ทั้งหมดแบบไม่ซ้ำกัน ที่บอทรู้จัก 
        # (ใช้สร้าง Bag of Words)
        self.vocabulary = []

        # self.intents = []: เก็บรายชื่อ Tag หรือหมวดหมู่ ทั้งหมด 
        # (เช่น "greeting", "goodbye", "thanks")
        self.intents = []

        # self.intents_response = {}: ดิกชันนารีที่เก็บ รายการคำตอบ ของแต่ละ Tag 
        # เพื่อให้บอทสุ่มดึงไปตอบผู้ใช้ได้รวดเร็ว
        self.intents_response = {}

        # (Optional) เป็นส่วนที่ใช้เชื่อมต่อ Intent เข้ากับฟังก์ชันในโปรแกรม
        self.function_mappings = function_mappings

        # จองที่ไว้เก็บ Features (ข้อมูล Input ในรูปตัวเลข 0, 1 จาก Bag of Words)
        self.X = None

        # จองที่ไว้เก็บ Labels (ตัวเลขดัชนีของแต่ละ Intent)
        self.y = None

    @staticmethod
    def tokenize_lemmatize(text):

        # แปลงคำให้กลับไปเป็น "รากศัพท์"
        lemmatizer = nltk.WordNetLemmatizer()

        # Tokenization: คือการตัดประโยคยาวๆ ออกเป็นคำย่อยๆ (Tokens)
        words = nltk.word_tokenize(text)

        # นำคำที่ได้ไปหาต้นแบบรากศัพท์ตามที่เตรียมไว้
        words = [lemmatizer.lemmatize(word.lower()) for word in words]

        return words
    
    # bag_of_words คือขั้นตอนการทำ Feature Extraction เพื่อเปลี่ยน "รายการคำศัพท์" (ซึ่งคอมพิวเตอร์คำนวณไม่ได้) 
    # ให้กลายเป็น "ตัวเลข (Vector)" ที่มีความยาวคงที่ เพื่อส่งให้ Neural Network
    def bag_of_words(self, words):

        # self.vocabulary: คือคลังคำศัพท์ทั้งหมดที่บอทเคยเรียนรู้มาจากไฟล์ JSON (เช่น ถ้าบอทรู้จักคำทั้งหมด 100 คำ ลิสต์นี้จะมีสมาชิก 100 ตัว)
        # for word in self.vocabulary: คือการวนลูปตรวจสอบทีละคำในคลังคำศัพท์ทั้งหมดตามลำดับ
        # return One-hot Encoding หรือ Bag of Words Vector

        return [1 if word in words else 0 for word in self.vocabulary]
    
    # เปลี่ยนข้อมูลจากไฟล์ JSON ให้กลายเป็นโครงสร้างข้อมูลที่พร้อมสำหรับการฝึกฝนโมเดล
    def parse_intents(self):
        lemmatizer = nltk.WordNetLemmatizer()

        if os.path.exists(self.intents_path):
            with open(self.intents_path, 'r') as f:
                intents_data = json.load(f)

            for intent in intents_data['intents']:
                if intent['tag'] not in self.intents:

                    # เก็บชื่อหมวดหมู่ (Tag) ทั้งหมดไว้ในลิสต์ เช่น ["greeting", "goodbye"]
                    self.intents.append(intent['tag'])

                    # สร้างฐานข้อมูลคำตอบไว้ในดิกชันนารี เพื่อให้บอทเรียกใช้ได้ทันทีเมื่อจำแนก Tag ได้แล้ว
                    self.intents_response[intent['tag']] = intent['responses']

                for pattern in intent['patterns']:

                    # นำประโยคมาตัดคำและแปลงเป็นรากศัพท์ 
                    pattern_words = self.tokenize_lemmatize(pattern)

                    # เก็บคำศัพท์ใหม่ๆ ที่เจอเพิ่มเข้าไปในคลังคำศัพท์รวม
                    self.vocabulary.extend(pattern_words)

                    # สร้างคู่หูข้อมูลเก็บไว้ในรูปแบบ (รายการคำศัพท์, ชื่อ Tag) 
                    # เพื่อบอกโมเดลว่า "ถ้าเจอคำกลุ่มนี้ ให้แปลว่าคือ Tag นี้"
                    self.documents.append((pattern_words, intent['tag']))

                # เพื่อให้ลำดับของคำใน Bag of Words คงที่เสมอ (เช่น ตำแหน่งที่ 0 คือคำว่า 'apple' เสมอ)
                self.vocabulary = sorted(set(self.vocabulary))

    # แปลงข้อมูลจาก "คู่คำศัพท์และหมวดหมู่" ให้กลายเป็น "ชุดตัวเลขเชิงคณิตศาสตร์" (Numerical Tensors)
    def prepare_data(self):

        # bags = []: ลิสต์สำหรับเก็บ Vector ของคำศัพท์ (ข้อมูลนำเข้า หรือ Input/Features)
        bags = []

        # indices = []: ลิสต์สำหรับเก็บตัวเลขดัชนีของหมวดหมู่ (คำเฉลย หรือ Output/Labels)
        indices = []

        for document in self.documents:

            # words = document[0]: ดึงรายการคำศัพท์ที่ตัดแล้วออกมา
            words = document[0]

            # bag = self.bag_of_words(words): เรียกฟังก์ชัน bag_of_words เพื่อเปลี่ยนรายการคำให้เป็นแถวของตัวเลข 0 และ 1 
            # (เช่น [0, 1, 0, 0, 1])
            bag = self.bag_of_words(words)

            # ทำ Label Encoding โดยการเปลี่ยนชื่อ Tag (เช่น "greeting") ให้กลายเป็นตัวเลขดัชนี
            intent_index = self.intents.index(document[1])

            # bags.append(bag) และ indices.append(intent_index): เก็บค่าที่แปลงได้ลงในลิสต์รวม
            bags.append(bag)
            indices.append(intent_index)

        # self.X = np.array(bags): แปลงลิสต์ของ Bag of Words ทั้งหมดให้เป็น Matrix ขนาดใหญ่ เพื่อใช้เป็นตัวแปรต้น (X)
        self.X = np.array(bags)

        # self.y = np.array(indices): แปลงลิสต์ของดัชนีให้เป็น Vector ของตัวเลขเฉลย เพื่อใช้เป็นตัวแปรตาม (y)
        self.y = np.array(indices)

    # train_model คือขั้นตอนการสอนบอทให้เรียนรู้ (Training Phase) โดยนำข้อมูลตัวเลขที่เตรียมไว้มาผ่านกระบวนการทางคณิตศาสตร์
    # เพื่อให้โมเดลจดจำความสัมพันธ์ระหว่างคำถามและคำตอบ
    def train_model(self, batch_size, lr, epochs):

        # torch.tensor(...): แปลงข้อมูลจาก NumPy ให้เป็น PyTorch Tensors 
        # เพื่อให้สามารถประมวลผลด้วย GPU หรือคำนวณ Gradient ได้
        X_tensor = torch.tensor(self.X, dtype=torch.float32)
        y_tensor = torch.tensor(self.y, dtype=torch.long)

        # TensorDataset: นำ X (คำถาม) และ y (เฉลย) มามัดรวมกันเป็นคู่ๆ
        dataset = TensorDataset(X_tensor, y_tensor)

        # DataLoader: ตัวจัดการการป้อนข้อมูล โดยจะแบ่งข้อมูลเป็นกลุ่มเล็กๆ (batch_size) 
        # และทำการสลับข้อมูล (shuffle) ในทุกรอบ เพื่อให้โมเดลไม่จำลำดับการป้อนข้อมูล PyTorch DataLoader Documentation
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # self.model = ChatbotModel(...): สร้างโครงสร้าง Neural Network โดยกำหนดขนาด 
        # Input ตามจำนวนคำศัพท์ที่มี และ Output ตามจำนวนหมวดหมู่ (Intents)
        self.model = ChatbotModel(self.X.shape[1], len(self.intents))

        # criterion = nn.CrossEntropyLoss(): ฟังก์ชันวัดค่าความผิดพลาด (Loss Function) 
        # สำหรับงานจำแนกประเภท (Classification)
        criterion = nn.CrossEntropyLoss()

        # optimizer = optim.Adam(...): อัลกอริทึมที่จะทำหน้าที่ปรับค่าน้ำหนักของโมเดล โดยใช้ค่า lr (Learning Rate) 
        # เป็นตัวกำหนดความเร็วในการก้าวเดิน Adam Optimizer Reference
        optimizer = optim.Adam(self.model.parameters(), lr=lr)

        for epoch in range(epochs):
            running_loss = 0.0

            for batch_X, batch_y in loader:

                # optimizer.zero_grad(): ล้างค่าความผิดพลาดเก่าทิ้งก่อนเริ่มรอบใหม่
                optimizer.zero_grad()

                # outputs = self.model(batch_X): ให้บอทลองทายคำตอบจากข้อมูลที่ได้รับ (Forward Pass)
                outputs = self.model(batch_X)

                # loss = criterion(outputs, batch_y): คำนวณดูว่าสิ่งที่บอททาย ต่างจากเฉลยมากน้อยแค่ไหน
                loss = criterion(outputs, batch_y)

                # loss.backward(): คำนวณทิศทางที่ต้องปรับปรุงค่าน้ำหนัก (Backpropagation)
                loss.backward()

                # optimizer.step(): ลงมือปรับค่าน้ำหนักในโมเดลจริงๆ
                optimizer.step()

                # running_loss += loss: สะสมค่าความผิดพลาดเพื่อนำมาพิมพ์รายงานผลในบรรทัดสุดท้าย
                running_loss += loss

            print(f"Epoch {epoch+1}: Loss: {running_loss / len(loader):.4f}")

    def save_model(self, model_path, dimension_path):

        # self.model.state_dict(): คำสั่งนี้จะดึงเฉพาะค่าน้ำหนัก (Weights) 
        # และค่า Bias ทั้งหมดที่โมเดลฝึกฝนจนสำเร็จออกมาในรูปแบบ Dictionary
        torch.save(self.model.state_dict(), model_path)

        with open(dimension_path, 'w') as f:
            json.dump({ 'input_size': self.X.shape[1], 'output_size': len(self.intents) }, f)

            # 'input_size': self.X.shape[1]: บันทึกว่าบอทตัวนี้รู้จักคำศัพท์ทั้งหมดกี่คำ (ขนาดของ Vocabulary)
            # 'output_size': len(self.intents): บันทึกว่าบอทมีหมวดหมู่คำตอบทั้งหมดกี่หมวดหมู่
            # json.dump(...): บันทึกค่าทั้งสองนี้ลงในไฟล์ JSON แยกต่างหาก

    def load_model(self, model_path, dimension_path):
        with open(dimension_path, 'r') as f:
            dimensions = json.load(f)

        # self.model = ChatbotModel(...): สร้างออบเจกต์โมเดลตัวเปล่าๆ ขึ้นมาตามขนาดที่อ่านได้จากไฟล์ Metadata
        self.model = ChatbotModel(dimensions['input_size'], dimensions['output_size'])

        self.model.load_state_dict(torch.load(model_path, weights_only=True))
        # torch.load(model_path, weights_only=True): โหลดค่าน้ำหนักที่บันทึกไว้ในไฟล์ .pth
        # self.model.load_state_dict(...): นำค่าน้ำหนักที่โหลดมาได้ ใส่เข้าไปในแต่ละเลเยอร์ของโมเดลตัวเปล่าที่เราเพิ่งสร้างขึ้น

    def process_message(self, input_message):

        # words = self.tokenize_lemmatize(input_message): นำประโยคที่คนพิมพ์มาตัดเป็นคำและหาต้นแบบรากศัพท์ 
        words = self.tokenize_lemmatize(input_message)

        # bag = self.bag_of_words(words): แปลงลิสต์คำศัพท์ให้กลายเป็น Vector ของตัวเลข 0 และ 1 ตามคลังศัพท์ที่บอทรู้จัก
        bag = self.bag_of_words(words)

        # bag_tensor = torch.tensor([bag], ...): แปลงข้อมูลเป็น PyTorch Tensor เพื่อส่งเข้าโมเดล
        bag_tensor = torch.tensor([bag], dtype=torch.float32)

        # self.model.eval(): เปลี่ยนโมเดลเข้าสู่ "โหมดใช้งานจริง" เพื่อปิดการทำงานของเลเยอร์สุ่มอย่าง Dropout PyTorch model.eval()
        self.model.eval()

        # with torch.no_grad():: สั่งไม่ให้คำนวณ Gradient เพื่อประหยัด RAM และทำให้บอทตอบกลับเร็วขึ้น
        with torch.no_grad():

            # predictions = self.model(bag_tensor): ส่งข้อมูลเข้าสมองกลเพื่อรับคะแนนความมั่นใจของแต่ละ Tag ออกมา
            predictions = self.model(bag_tensor)

        # torch.argmax(..., dim=1).item(): เลือกตำแหน่ง (Index) ของ Tag ที่ได้คะแนนสูงสุดเพียงอันเดียว
        predicted_class_index = torch.argmax(predictions, dim=1).item()

        # predicted_intent = self.intents[...]: แปลงตัวเลข Index กลับเป็นชื่อ Tag ที่มนุษย์เข้าใจ (เช่น "greeting")
        predicted_intent = self.intents[predicted_class_index]

        if self.function_mappings:
            if predicted_intent in self.function_mappings:
                self.function_mappings[predicted_intent]()

            if self.intents_response[predicted_intent]:
                return random.choice(self.intents_response[predicted_intent])
            else:
                return None

